Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [93]:
# This part is for logistic regression 
batch_size = 128
beta=0.006

graph = tf.Graph()
with graph.as_default():

    # Input data. At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)

  
    # Variables    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss =  tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) 
    loss = tf.reduce_mean(loss + beta  * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [94]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 34.636108
Minibatch accuracy: 12.5%
Validation accuracy: 14.3%
Minibatch loss at step 500: 1.460374
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 0.637752
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Test accuracy: 88.4%


In [123]:
#This part is for Neural Network
batch_size = 128
hidden_layer_size = 1024
#beta=0.006
graph = tf.Graph()
with graph.as_default():
    
    # Input data. At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    #input -> hidden layer -> output layer
    #Hidden Layer Variables
    hidden_layer_weights = tf.Variable( tf.truncated_normal([image_size * image_size,hidden_layer_size ]))
    hidden_layer_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_layer_output  = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases)
    
    # Output Layer Variables
    output_layer_weights = tf.Variable( tf.truncated_normal([hidden_layer_size, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    output_layer_output = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases
    
    
    # Training computation.  
    loss = tf.nn.softmax_cross_entropy_with_logits(output_layer_output, tf_train_labels)
    
    #Regularization: L2
    #Here I learned from the work of  github.com/Arn-O, he tried to update loss with both the hidden layer and the ouput layer.
    loss = tf.reduce_mean( loss + beta_regul * (tf.nn.l2_loss(hidden_layer_weights)+tf.nn.l2_loss(output_layer_weights) )) 
    
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output_layer_output)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_layer_weights) + hidden_layer_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_layer_weights) + hidden_layer_biases)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)

In [124]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_regul : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 719.395508
Minibatch accuracy: 6.2%
Validation accuracy: 32.5%
Minibatch loss at step 500: 196.310791
Minibatch accuracy: 80.5%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 114.005936
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.790779
Minibatch accuracy: 82.8%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 41.271774
Minibatch accuracy: 90.6%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 25.191772
Minibatch accuracy: 85.9%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.352528
Minibatch accuracy: 93.0%
Validation accuracy: 86.5%
Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [131]:
#This part is for Neural Network
batch_size = 128
hidden_layer_size = 1024
#beta=0.006
graph = tf.Graph()
with graph.as_default():
    
    # Input data. At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    #input -> hidden layer -> output layer
    #Hidden Layer Variables
    hidden_layer_weights = tf.Variable( tf.truncated_normal([image_size * image_size,hidden_layer_size ]))
    hidden_layer_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_layer_output  = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases)
    
    # Output Layer Variables
    output_layer_weights = tf.Variable( tf.truncated_normal([hidden_layer_size, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    output_layer_output = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases
    
    
    # Training computation.  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer_output, tf_train_labels))
    
    #Regularization: No
    
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output_layer_output)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_layer_weights) + hidden_layer_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_layer_weights) + hidden_layer_biases)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)

In [132]:
num_steps = 3001
num_batches=3
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        #The control of the randomization is very important.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = step % ( num_batches)
        
        #Generate a batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 309.821167
Minibatch accuracy: 10.2%
Validation accuracy: 28.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.5%
Test accuracy: 74.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [137]:
#This part is for Neural Network
batch_size = 128
hidden_layer_size = 1024
graph = tf.Graph()
with graph.as_default():
    
    # Input data. At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    #input -> hidden layer -> output layer
    #Hidden Layer Variables
    hidden_layer_weights = tf.Variable( tf.truncated_normal([image_size * image_size,hidden_layer_size ]))
    hidden_layer_biases = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_layer_output  = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_layer_weights) + hidden_layer_biases)
    
    #Regularization: Drop out 
    #Remember: Drop out always happens between layers
    hidden_layer_output_after_drop = tf.nn.dropout(hidden_layer_output, 0.6)
    
    # Output Layer Variables
    output_layer_weights = tf.Variable( tf.truncated_normal([hidden_layer_size, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    output_layer_output = tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases
    
    
    # Training computation.  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer_output, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(output_layer_output)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_layer_weights) + hidden_layer_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)
    
    hidden_layer_output = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_layer_weights) + hidden_layer_biases)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_layer_output, output_layer_weights) + output_layer_biases)

In [138]:
num_steps = 3001
num_batches=3
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        #The control of the randomization is very important.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = step % ( num_batches)
        
        #Generate a batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 381.021790
Minibatch accuracy: 10.9%
Validation accuracy: 34.6%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.9%
Test accuracy: 71.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [171]:
#This part is for Neural Network
batch_size = 128

hidden_layer1_size = 1024
hidden_layer2_size = 256
hidden_layer3_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. At run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0)
    
    #input -> hidden layer -> output layer
    #Hidden Layer 1
    hidden_layer1_weights = tf.Variable( tf.truncated_normal([image_size * image_size,hidden_layer1_size ], stddev=np.sqrt(2.0 / (image_size * image_size))))
    hidden_layer1_biases = tf.Variable(tf.zeros([hidden_layer1_size]))
    hidden_layer1_output  = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_layer1_weights) + hidden_layer1_biases)
    hidden_layer1_output_after_drop = tf.nn.dropout(hidden_layer1_output, 0.5)
    #Hidden Layer 2
    hidden_layer2_weights = tf.Variable( tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev=np.sqrt(2.0 / hidden_layer1_size)))
    hidden_layer2_biases = tf.Variable(tf.zeros([hidden_layer2_size]))
    hidden_layer2_output  = tf.nn.relu(tf.matmul(hidden_layer1_output_after_drop, hidden_layer2_weights) + hidden_layer2_biases)
    hidden_layer2_output_after_drop = tf.nn.dropout(hidden_layer2_output, 0.5)
    #Hidden Layer 3
    hidden_layer3_weights = tf.Variable( tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev=np.sqrt(2.0 / hidden_layer2_size)))
    hidden_layer3_biases = tf.Variable(tf.zeros([hidden_layer3_size]))
    hidden_layer3_output  = tf.nn.relu(tf.matmul(hidden_layer2_output_after_drop, hidden_layer3_weights) + hidden_layer3_biases)
    hidden_layer3_output_after_drop = tf.nn.dropout(hidden_layer3_output, 0.5)
    
    # Output Layer Variables
    output_layer_weights = tf.Variable( tf.truncated_normal([hidden_layer3_size, num_labels],stddev=np.sqrt(2.0 / hidden_layer3_size)))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    output_layer_output = tf.matmul(hidden_layer3_output_after_drop, output_layer_weights) + output_layer_biases
      
    # Training computation.  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer_output, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #Predictions for the training data.
    train_prediction = tf.nn.softmax(output_layer_output)
    #Predictions for the validation data.
    hidden_layer1_output = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_layer1_weights) + hidden_layer1_biases)
    hidden_layer2_output = tf.nn.relu(tf.matmul(hidden_layer1_output, hidden_layer2_weights) + hidden_layer2_biases)
    hidden_layer3_output = tf.nn.relu(tf.matmul(hidden_layer2_output, hidden_layer3_weights) + hidden_layer3_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(hidden_layer3_output, output_layer_weights) + output_layer_biases)
    #Predictions for the test data.
    hidden_layer1_output = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_layer1_weights) + hidden_layer1_biases)
    hidden_layer2_output = tf.nn.relu(tf.matmul(hidden_layer1_output, hidden_layer2_weights) + hidden_layer2_biases)
    hidden_layer3_output = tf.nn.relu(tf.matmul(hidden_layer2_output, hidden_layer3_weights) + hidden_layer3_biases)
    test_prediction = tf.nn.softmax( tf.matmul(hidden_layer3_output, output_layer_weights) + output_layer_biases)

In [172]:
num_steps = 18001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.646016
Minibatch accuracy: 7.0%
Validation accuracy: 16.9%
Minibatch loss at step 500: 0.516561
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.447040
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 1500: 0.554450
Minibatch accuracy: 82.0%
Validation accuracy: 85.1%
Minibatch loss at step 2000: 0.352772
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 0.604017
Minibatch accuracy: 84.4%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 0.336978
Minibatch accuracy: 90.6%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 0.605498
Minibatch accuracy: 83.6%
Validation accuracy: 86.3%
Minibatch loss at step 4000: 0.730647
Minibatch accuracy: 79.7%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 0.321853
Minibatch accuracy: 89.8%
Validation accuracy: 87.5%
Minibatch loss at step 5000: 0.578736
Minibatch accuracy: 82.0%
Validation accuracy